## Logistic Regression

2018-04-25：更新为新版本代码

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

定义超参数

In [2]:
batch_size = 32
learning_rate = 1e-3
num_epoches = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # whether GPU is supportted

载入/下载训练集 MNIST 手写数字训练集

In [3]:
train_dataset = datasets.MNIST('../_data/mnist', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST('../_data/mnist', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

定义 Logistic Regression 模型

In [4]:
class Logistic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logistic_Regression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)
        
    def forward(self, x):
        out = self.logistic(x)
        return out

In [5]:
model = Logistic_Regression(28 * 28, 10).to(device) # pictures' size are 28x28

逻辑斯蒂回归与线性回归的区别可以在 loss 函数的定义上，Logistic 回归的损失函数为交叉熵损失函数，即

$$L_H(\mathbf x,\mathbf z)=-\sum_{k=1}^dx_k\log z_k+(1-x_k)\log(1-z_k).$$

优化器与前者一致。

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

开始训练

In [7]:
for epoch in range(num_epoches):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1) # 将图片展开成 28x28
        img, label = img.to(device), label.to(device)
        # forward propagation
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()
        running_acc += num_correct
        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epoches}] Loss: {running_loss/(batch_size*i):.6f}, Acc: {running_acc/(batch_size*i):.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/(len(train_dataset)):.6f}, Acc: {running_acc/len(train_dataset):.6f}')
    model.eval()
    eval_loss = 0.0
    eval_acc = 0.0
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        img, label = img.to(device), label.to(device)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()
        eval_acc += num_correct
    print(f'Test Loss: {eval_loss/(len(test_dataset)):.6f}, Acc: {eval_acc/(len(test_dataset)):.6f}')
    print(f'Time: {time.time() - since:.1f}')            

**********
epoch 1
[1/100] Loss: 2.189628, Acc: 0.269896
[1/100] Loss: 2.054564, Acc: 0.458542
[1/100] Loss: 1.941132, Acc: 0.548090
[1/100] Loss: 1.842568, Acc: 0.600078
[1/100] Loss: 1.755603, Acc: 0.636354
[1/100] Loss: 1.680480, Acc: 0.662031
Finish 1 epoch, Loss: 1.663267, Acc: 0.667633
Test Loss: 1.212778, Acc: 0.808900
Time: 7.1
**********
epoch 2
[2/100] Loss: 1.192363, Acc: 0.806250
[2/100] Loss: 1.158313, Acc: 0.806927
[2/100] Loss: 1.129286, Acc: 0.806736
[2/100] Loss: 1.099719, Acc: 0.810391
[2/100] Loss: 1.075341, Acc: 0.812583
[2/100] Loss: 1.051600, Acc: 0.814635
Finish 2 epoch, Loss: 1.045255, Acc: 0.815400
Test Loss: 0.883197, Acc: 0.835800
Time: 9.2
**********
epoch 3
[3/100] Loss: 0.887939, Acc: 0.829479
[3/100] Loss: 0.879559, Acc: 0.829948
[3/100] Loss: 0.864300, Acc: 0.832396
[3/100] Loss: 0.851407, Acc: 0.834141
[3/100] Loss: 0.840726, Acc: 0.835562
[3/100] Loss: 0.831778, Acc: 0.835330
Finish 3 epoch, Loss: 0.828071, Acc: 0.836000
Test Loss: 0.737549, Acc: 0.850

保存模型

In [8]:
torch.save(model.state_dict(), 'save/02-logistic regression.pytorch')